In [1]:
from transformers import EncoderDecoderModel, AutoTokenizer, AutoModel, ModernBertModel, BertTokenizer, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from datasets import load_dataset, load_from_disk
import torch
from model.tokenizer import OptiSeqTokenizer

ModuleNotFoundError: No module named 'datasets'

In [2]:
model_path = "/home/xucong24/Compiler/models/opti_seq_base"
modernbert_tokenizer = AutoTokenizer.from_pretrained('/home/xucong24/Compiler/work_dirs/modernbert/20250718_044355/final_model')
opti_seq_tokenizer = OptiSeqTokenizer.from_pretrained('/home/xucong24/Compiler/checkpoints/opti_seq_tokenizer')

In [3]:
model = EncoderDecoderModel.from_pretrained('/home/xucong24/Compiler/work_dirs/llvm_opt_gen/20250718_171657/checkpoint-11319')

In [4]:
model

EncoderDecoderModel(
  (encoder): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      )
      (1-21)

In [5]:
dataset = load_from_disk('/home/xucong24/Compiler/datasets/llvm_opti_seq')

Loading dataset from disk:   0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
sample = dataset['test'][0]

In [7]:
inputs = modernbert_tokenizer(sample['LLVM_IR'], return_tensors="pt", truncation=True, padding='max_length', max_length=512)

In [8]:
generated = model.generate(**inputs)

Setting `pad_token_id` to `eos_token_id`:127 for open-end generation.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.53.0. You should pass an instance of `Cache` instead, e.g. `past_key_values=DynamicCache.from_legacy_cache(past_key_values)`.


In [9]:
generated

tensor([[126,  26,  15,  11,  76, 113, 113,  45,  40,  34,  10,  36,  65,  95,
          28,  30, 107,   4,  69,  19,  74]])

In [10]:
opti_seq_tokenizer.decode(generated[0])

'-dse -constmerge -callsite-splitting -loop-unroll-and-jam -separate-const-offset-from-gep -separate-const-offset-from-gep -ipconstprop -globaldce -float2int -simplifycfg -inline -loop-idiom -name-anon-globals -div-rem-pairs -early-cse -rpo-functionattrs -always-inline -loop-predication -coro-elide -loop-sink'

In [11]:
sample['LLVM_IR']

'; ModuleID = \'-\'\nsource_filename = "-"\ntarget datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-unknown-linux-gnu"\n\n%union.U0 = type { i8* }\n\n@.str = private unnamed_addr constant [2 x i8] c"1\\00", align 1\n@g_13 = internal global [7 x i32] [i32 7, i32 1, i32 7, i32 7, i32 1, i32 7, i32 7], align 16\n@.str.1 = private unnamed_addr constant [8 x i8] c"g_13[i]\\00", align 1\n@.str.2 = private unnamed_addr constant [14 x i8] c"index = [%d]\\0A\\00", align 1\n@g_17 = internal global i32 1097990984, align 4\n@.str.3 = private unnamed_addr constant [5 x i8] c"g_17\\00", align 1\n@g_19 = internal global i8 1, align 1\n@.str.4 = private unnamed_addr constant [5 x i8] c"g_19\\00", align 1\n@g_45 = internal global [3 x [3 x [3 x i8]]] [[3 x [3 x i8]] [[3 x i8] c"\\00_\\00", [3 x i8] c"\\F9_\\9B", [3 x i8] c"\\07\\00\\01"], [3 x [3 x i8]] [[3 x i8] c"\\D6\\07\\07", [3 x i8] c"\\07F\\B8", [3 x i8] c"\\F9M\\B8"], [3 x [3 x i8]] 

In [13]:
from utils.llvm import calc_reward
calc_reward(sample['LLVM_IR'], opt_flags=opti_seq_tokenizer.decode(generated[0]), llvm_path='/home/xucong24/.local/share/compiler_gym/llvm-v0/bin')

IR指令数量: 15428, 优化后指令数量: 16867, O3优化后指令数量: 1324, 优化奖励: -0.10202779353374929


-0.10202779353374929

## 模型训练

In [ ]:
input_ids = modernbert_tokenizer(sample['LLVM_IR'], return_tensors="pt", truncation=True, padding='max_length', max_length=512).input_ids
labels = opti_seq_tokenizer("-adce -alignment-from-assumptions -alignment-from-assumptions")['input_ids']
outputs = model(input_ids=input_ids, labels=labels)
loss, logits = outputs.loss, outputs.logits

/home/xucong24/Compiler/.venv/lib/python3.12/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:557: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/home/xucong24/Compiler/.venv/lib/python3.12/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:127 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.53.0. You should pass an instance of `Cache` instead, e.g. `past_key_values=DynamicCache.from_legacy_cache(past_key_values)`.


In [78]:
decoder_tokenizer.decode(generated[0])

'-gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist -gvn-hoist'

In [82]:
dataset = load_from_disk('/home/xucong24/Compiler/datasets/llvm_opti_seq')   

Loading dataset from disk:   0%|          | 0/33 [00:00<?, ?it/s]

In [86]:
test_dataset = dataset["train"].select(range(100))

In [89]:
len(test_dataset[0]['Commandline'].split())

325

In [90]:
test_dataset    

Dataset({
    features: ['Benchmark', 'CpuInfo', 'IrInstructionCountO0', 'IrInstructionCountO3', 'IrInstructionCountOz', 'InstCount', 'Autophase', 'Reward', 'LLVM_IR', 'Commandline'],
    num_rows: 100
})

In [91]:
def tokenize_fn(examples):
    llvm_ir = examples['LLVM_IR']
    input_ids = encoder_tokenizer(
        llvm_ir, 
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors='pt'
    ).input_ids

    commandline = examples['Commandline']
    labels = decoder_tokenizer(
        commandline,
        truncation=True,
        padding=True,
        max_length=512,
    )['input_ids']

    return {
        'input_ids': input_ids,
        'labels': labels
    }


test_dataset = test_dataset.map(
    tokenize_fn, 
    batched=True,
    remove_columns=['Benchmark', 'CpuInfo', 'IrInstructionCountO0', 'IrInstructionCountO3', 'IrInstructionCountOz', 'InstCount', 'Autophase', 'Reward', 'LLVM_IR', 'Commandline']
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    "google-bert/bert-base-uncased", "google-bert/bert-base-uncased"
)  # initialize Bert2Bert from pre-trained checkpoints

# training
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

input_ids = tokenizer("This is a really long text", return_tensors="pt").input_ids
labels = tokenizer("This is the corresponding summary", return_tensors="pt").input_ids
outputs = model(input_ids=input_ids, labels=labels)
loss, logits = outputs.loss, outputs.logits

# save and load from pretrained
model.save_pretrained("bert2bert")
model = EncoderDecoderModel.from_pretrained("bert2bert")

# generation
generated = model.generate(input_ids)